<a href="https://colab.research.google.com/github/lia-ang/BME3053C_ECG_Project/blob/main/final_project_ecg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Group Project: Angela Liang, Ece Sarioglu, Ankitha Nath, Siri Yarrapatni, Vivian Lam, Alana Young

Repositories: https://github.com/huggingface/transformers/tree/main/src/transformers/models/longformer
https://github.com/allenai/longformer

Dataset: https://physionet.org/content/ecg-arrhythmia/1.0.0/

In [1]:
# Import libraries for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import libraries for signal processing
from scipy.signal import butter, lfilter, welch

# Import libraries for machine learning and evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Import PyTorch and Hugging Face Transformers for Longformer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import LongformerTokenizer, LongformerModel, LongformerForSequenceClassification

# Import additional utilities
import os
import warnings
warnings.filterwarnings('ignore')

# For reproducibility
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
# Install required libraries
pip install wfdb numpy pandas transformers torch datasets scikit-learn